In [12]:
import re

In [13]:
# Let's read the contents of the file "sample_data/stderr.txt" in Python.
file_path = 'sample_data/stderr.txt'

# Reading the file and storing its contents in a variable.
with open(file_path, 'r') as file:
    sample_text = file.read()

sample_text

"\n> spc-llamathon@1.0.0 test\n> jest example.test.js\n\nFAIL ./example.test.js\n  add function\n    ✓ should add two positive numbers correctly (1 ms)\n    ✓ should add two negative numbers correctly\n    ✓ should add a positive and a negative number correctly\n    ✓ should add two zeros correctly\n    ✓ should add a number and zero correctly\n    ✓ should add two decimal numbers correctly\n    ✓ should add a decimal and an integer number correctly\n    ✓ should add two large numbers correctly\n    ✓ should return NaN when adding NaN\n    ✓ should return Infinity when adding Infinity\n    ✓ should return -Infinity when adding -Infinity (1 ms)\n  multiply\n    ✕ should multiply two positive numbers correctly\n    ✕ should multiply two negative numbers correctly\n    ✕ should multiply a positive and a negative number correctly\n    ✕ should multiply two zero numbers correctly\n    ✕ should multiply a number with zero correctly\n    ✕ should multiply a number with one correctly\n    ✕ sh

In [33]:
# Regular expression to recognize the start of a block and the end of the block
block_pattern = re.compile(r'(  .+?)(?:\n\s*\n)', re.DOTALL)

# Finding the first block in the sample text
match = block_pattern.search(sample_text)

# Get and print the first block
block = match.group(1)
print(block)

  add function
    ✓ should add two positive numbers correctly (1 ms)
    ✓ should add two negative numbers correctly
    ✓ should add a positive and a negative number correctly
    ✓ should add two zeros correctly
    ✓ should add a number and zero correctly
    ✓ should add two decimal numbers correctly
    ✓ should add a decimal and an integer number correctly
    ✓ should add two large numbers correctly
    ✓ should return NaN when adding NaN
    ✓ should return Infinity when adding Infinity
    ✓ should return -Infinity when adding -Infinity (1 ms)
  multiply
    ✕ should multiply two positive numbers correctly
    ✕ should multiply two negative numbers correctly
    ✕ should multiply a positive and a negative number correctly
    ✕ should multiply two zero numbers correctly
    ✕ should multiply a number with zero correctly
    ✕ should multiply a number with one correctly
    ✕ should multiply large numbers correctly
    ✕ should multiply decimal numbers correctly
    ✕ should r

In [37]:
res = {}
latest_suite = None

def handle_block(block):
    global latest_suite
    lines = block.split('\n')

    for line in lines:
        # detect if new suit
        if not line.startswith('   '):
            suit_name = line.strip()
            res[suit_name] = {}
            latest_suite = suit_name
            continue
        
        status = line[4]
        description = line[6:]

        # Regular expression to match and remove the " (...)" part at the end
        pattern = re.compile(r' \(.+\)$')

        # Remove the " (...)" part if present
        description = pattern.sub('', description)

        res[latest_suite][description] = False if status == '✕' else True

handle_block(block)
res

{'add function': {'should add two positive numbers correctly': True,
  'should add two negative numbers correctly': True,
  'should add a positive and a negative number correctly': True,
  'should add two zeros correctly': True,
  'should add a number and zero correctly': True,
  'should add two decimal numbers correctly': True,
  'should add a decimal and an integer number correctly': True,
  'should add two large numbers correctly': True,
  'should return NaN when adding NaN': True,
  'should return Infinity when adding Infinity': True,
  'should return -Infinity when adding -Infinity': True},
 'multiply': {'should multiply two positive numbers correctly': False,
  'should multiply two negative numbers correctly': False,
  'should multiply a positive and a negative number correctly': False,
  'should multiply two zero numbers correctly': False,
  'should multiply a number with zero correctly': False,
  'should multiply a number with one correctly': False,
  'should multiply large num